In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/fundos-imobiliarios/busca-avancada')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/funds.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [36]:
fundinvest = pd.read_csv('funds.csv', sep=';',  decimal=',', thousands='.', encoding="utf-8" )
#fiis = pd.DataFrame(funds)


fundinvest = fundinvest.loc[:, ['TICKER', 'PRECO', 'GESTAO' , 'DY', 'P/VP', 'PERCENTUAL EM CAIXA', 
                        'N COTISTAS', 'CAGR DIVIDENDOS 3 ANOS', ' CAGR VALOR CORA 3 ANOS', 
                        'LIQUIDEZ MEDIA DIARIA', 'PATRIMONIO', 'VALOR PATRIMONIAL COTA', 
                        ' N COTAS', 'ULTIMO DIVIDENDO']]

fundinvest.fillna(value=0, inplace=True)

fundinvest.rename(columns={'PERCENTUAL EM CAIXA': "PERC_CX" }, inplace=True)
fundinvest.rename(columns={'CAGR DIVIDENDOS 3 ANOS': "DY_CAGR_3ANOS" }, inplace=True)
fundinvest.rename(columns={' CAGR VALOR CORA 3 ANOS': "VLR_CAGR_3ANOS" }, inplace=True)
fundinvest.rename(columns={'LIQUIDEZ MEDIA DIARIA': "LIQ_DIARIA" }, inplace=True)
fundinvest.rename(columns={'VALOR PATRIMONIAL COTA': "VLR_PATR_COTA" }, inplace=True)
fundinvest.rename(columns={'ULTIMO DIVIDENDO': "ULT_DIV" }, inplace=True)

fundinvest.rename(columns={'TICKER': "TICKERS" }, inplace=True)


fundinvest = fundinvest.loc[fundinvest['DY'] > 0].loc[
    fundinvest['DY_CAGR_3ANOS'] >= 0.01].loc[
    fundinvest['VLR_CAGR_3ANOS'] >= 0.01].loc[
    fundinvest['N COTISTAS'] >= 1000].loc[
    fundinvest['LIQ_DIARIA'] >= 100].loc[
    fundinvest['ULT_DIV'] >= 0.1]

len(fundinvest.index)

25

In [37]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = fundinvest['TICKERS']
list_funds = ticker.to_list()

indicators_list_funds = []
for funds in list_funds:

    #Criando url
    url = f"https://statusinvest.com.br/fundos-imobiliarios/{funds}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        NAME = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/h1/small').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[3]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[5]/div/div[1]/strong').text
        TIPO = navegador.find_element(By.XPATH, '//*[@id="fund-section"]/div/div/div[2]/div/div[5]/div/div/div/strong').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="fund-section"]/div/div/div[2]/div/div[6]/div/div/strong').text
        
        

        #Criando dict
        dicionario = {  "funds": funds,
                        "NAME": NAME,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "TIPO": TIPO,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        indicators_list_funds.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()

In [94]:
funds = pd.DataFrame.from_dict(indicators_list_funds)

funds = funds.replace('-', '')
funds = funds.replace('-%', '')
funds['MIN_MES'] = (funds['MIN_MES']).str.lstrip('R$ ')
funds['MAX_MES'] = (funds['MAX_MES']).str.lstrip('R$ ')
funds['MIN_MES'] = (funds['MIN_MES']).str.strip('- ')
funds['MAX_MES'] = (funds['MAX_MES']).str.strip('- ')
funds['VAL_12M'] = (funds['VAL_12M']).str.rstrip('%')
funds['VAL_MES'] = (funds['VAL_MES']).str.rstrip('%')

funds.to_csv('list_funds.csv')

list_funds = pd.read_csv("list_funds.csv", sep=',', decimal=',', thousands='.', encoding="utf-8" )

list_funds.fillna(value=0, inplace=True)
list_funds = list_funds.drop(['Unnamed: 0'],axis=1)
list_funds.rename(columns={'funds': "TICKERS" }, inplace=True)

invest_funds = pd.merge(fundinvest, list_funds, on='TICKERS')

invest_funds['UP/DOWNSIDE'] = (invest_funds['VLR_PATR_COTA'] / invest_funds['PRECO'])-1

invest_funds = invest_funds[['TICKERS', 'NAME',  'SEGMENTO', 'GESTAO', 'N COTISTAS', 
                             'PATRIMONIO', ' N COTAS', 'VLR_PATR_COTA', 'UP/DOWNSIDE', 'PRECO', 
                             'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S', 
                             'VAL_12M', 'ULT_DIV', 'DY', 'DY_CAGR_3ANOS', 'P/VP', 
                             'PERC_CX',  'VLR_CAGR_3ANOS', 'LIQ_DIARIA', ]]

invest_funds = invest_funds.loc[invest_funds['DY'] > 0].loc[
    invest_funds['DY_CAGR_3ANOS'] >= 0.01].loc[
    invest_funds['VLR_CAGR_3ANOS'] >= 0.01].loc[
    invest_funds['N COTISTAS'] >= 1000].loc[
    invest_funds['LIQ_DIARIA'] >= 100].loc[
    invest_funds['ULT_DIV'] >= 0.1].loc[
    invest_funds['UP/DOWNSIDE'] > 0]

# def formatar (valor):
#     return "{:.2}".format(float(valor))

invest_funds['PATRIMONIO'] = invest_funds['PATRIMONIO'] / 1000000
invest_funds[' N COTAS'] = invest_funds[' N COTAS'] / 1000
invest_funds['N COTISTAS'] = invest_funds['N COTISTAS'] / 1000



invest_funds

,TICKERS,NAME,SEGMENTO,GESTAO,N COTISTAS,PATRIMONIO,N COTAS,VLR_PATR_COTA,UP/DOWNSIDE,PRECO,...,MIN_52S,MAX_52S,VAL_12M,ULT_DIV,DY,DY_CAGR_3ANOS,P/VP,PERC_CX,VLR_CAGR_3ANOS,LIQ_DIARIA
0,ARRI11,ÁTRIO REIT RECEBÍVEIS IMOBILIÁRIOS,Títulos e Valores Mobiliários,Ativa,27.459,117.478339,13056.237,9.00,0.021566,8.81,...,8.06,9.43,-1.01,0.10,15.32,3.73,0.98,2.49,2.84,300934.06
1,BMLC11,BM BRASCAN LAJES CORPORATIVAS,Híbrido,Ativa,1.505,110.629236,998.405,110.81,0.045279,106.01,...,98.00,108.99,6.01,0.76,11.42,20.48,0.96,17.32,4.63,34252.61
2,BTCR11,BTG PACTUAL CRÉDITO IMOBILIÁRIO,Títulos e Valores Mobiliários,Ativa,16.934,458.252684,4810.097,95.27,0.082614,88.00,...,85.45,97.79,-8.42,0.90,2.05,32.78,0.92,8.00,0.89,652774.65
3,EURO11,EUROPAR,Logística,Passiva,2.429,124.873196,383.936,325.24,0.319432,246.50,...,218.31,250.00,6.72,1.88,9.06,10.36,0.76,1.54,4.81,40495.65
4,FCFL11,CAMPUS FARIA LIMA,Outros,Ativa,3.634,403.741892,3474.010,116.22,0.000172,116.20,...,98.92,131.00,-7.35,0.78,8.22,9.63,1.00,0.67,1.39,159354.39
8,HGPO11,CSHG PRIME OFFICES FDO INV IMOB,Lajes Corporativas,Passiva,15.267,524.362205,1753.057,299.11,0.088861,274.70,...,238.00,297.00,4.68,1.60,6.99,13.65,0.92,0.61,7.36,591018.61
10,HSML11,HSI MALL FDO INV IMOB,Shoppings,Ativa,160.995,1503.214578,15780.613,95.26,0.007936,94.51,...,74.09,96.92,3.86,0.74,9.12,21.13,0.99,5.23,0.56,2906062.03
12,JRDM11,SHOPPING JARDIM SUL,Shoppings,Passiva,4.001,224.143181,2522.968,88.84,0.034226,85.90,...,76.56,87.93,-0.58,0.40,7.36,28.90,0.97,0.41,3.57,610682.00
13,KNCR11,KINEA RENDIMENTOS IMOBILIÁRIOS,Títulos e Valores Mobiliários,Ativa,257.717,5758.807700,57097.087,100.86,0.018685,99.01,...,93.00,102.40,-2.14,1.20,13.90,49.84,0.98,6.33,3.30,11436523.00
14,MALL11,MALLS BRASIL PLURAL,Shoppings,Ativa,124.481,1092.773095,9124.659,119.76,0.017589,117.69,...,97.02,119.69,8.80,0.85,8.06,15.68,0.98,2.11,7.95,2893946.61
